###  firefox 브라우저를 통해 수집 대상 페이지 접근
* 로그인
* 모바일 페이지로 이동
* 검색어 입력
* 결과 더보기 클릭

## Selenium 구동

In [571]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time
import pandas as pd

binary = FirefoxBinary('C:\Program Files\Mozilla Firefox\firefox.exe')
driver = webdriver.Firefox(firefox_binary=binary)
time.sleep(1)

 # 창 크기 최대화
driver.maximize_window()
time.sleep(1)

In [572]:
def starter(cafe_id):
    cafe_info = {'10050813':{'id':'OOOO','pw':'00000000'},
                '10094499':{'id':'OOOO','pw':'00000000'},
                '10298136':{'id':'OOOO','pw':'00000000'}}

    # 로그인 페이지로 이동
    driver.get('https://nid.naver.com/nidlogin.login')
    time.sleep(1)

    # 로그인 정보 입력
    driver.find_element_by_id('id').send_keys(cafe_info[cafe_id]['id'])
    driver.find_element_by_id('pw').send_keys(cafe_info[cafe_id]['pw'])

    # 로그인 버튼 클릭
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(1)
                 
    # 키워드 검색 페이지로 이동
    driver.get('https://m.cafe.naver.com/ArticleSearchList.nhn?search.clubid='+cafe_id)              
                 
def get_doc_list_with_keywords(search_keywords):
    
    doc_list = []
    for keyword in search_keywords:
        
        # 검색어 입력
        driver.find_element_by_id('searchQuery').clear()
        driver.find_element_by_id('searchQuery').send_keys(keyword)
        
        # 검색버튼 클릭
        driver.find_element_by_xpath('//*[@id="myCafeSearchForm"]/div/div[2]/a[2]').click()
        time.sleep(1)
        
        # 20개 더보기 버튼 클릭 
        index = 0
        while True:
            index += 1
            try:
                driver.find_element_by_css_selector('a[id="moreButton"]').click()
                print('click next page ' + '.'*index, end='\r')
                time.sleep(1)
            except NoSuchElementException as e:
                print('no more pages')
                break
        results = [{'url':a.get_attribute('href'),
                    'title':a.find_element_by_css_selector('h3').text,
                    'content':a.find_element_by_css_selector('div.post_area').text} for a in driver.find_elements_by_css_selector('ul.lst_section > li > a')]
        doc_list += results
        print('검색어:' + keyword + ' | 문서량:' + str(len(results)))
        
    return doc_list

## 게시글 리스트 수집

In [573]:
# 수집 대상 카페 정보 
cafe_ids = {'파우더룸':'10050813', '맘스홀릭':'10094499', '레몬테라스':'10298136'}

# 검색 키워드
search_keywords = ['기아소울','쏘울','Soul','스토닉','STONIC','코나','KONA','QM3',
                   '큐엠3','큐앰쓰리','큐엠쓰리','큐앰3','큐삼이','티볼리','TIVOLI','컨트리맨','BMW 미니']

# 로그인 및 수집 대상 카페 이동
starter(cafe_ids['레몬테라스'])

# 게시글 리스트 수집
doc_list = get_doc_list_with_keywords(search_keywords)
print('리스트 페이지 수집 완료')
df_doc_list = pd.DataFrame(doc_list)

no more pagesge .
검색어:기아소울 | 문서량:29
no more pagesge ..................
검색어:쏘울 | 문서량:376
no more pagesge ...............
검색어:Soul | 문서량:301
no more pages
검색어:스토닉 | 문서량:19
no more pages
검색어:STONIC | 문서량:0
no more pagesge ........................................
검색어:코나 | 문서량:805
no more pagesge ..........
검색어:KONA | 문서량:219
no more pagesge .......
검색어:QM3 | 문서량:154
no more pages
검색어:큐엠3 | 문서량:12
no more pages
검색어:큐앰쓰리 | 문서량:0
no more pages
검색어:큐엠쓰리 | 문서량:8
no more pages
검색어:큐앰3 | 문서량:0
no more pages
검색어:큐삼이 | 문서량:0
no more pagesge .........................................................................................
검색어:티볼리 | 문서량:1799
no more pages
검색어:TIVOLI | 문서량:10
no more pagesge ...
검색어:컨트리맨 | 문서량:62
no more pagesge .........
검색어:BMW 미니 | 문서량:182
리스트 페이지 수집 완료


In [574]:
df_doc_list = pd.DataFrame(doc_list)

In [575]:
df_doc_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3976 entries, 0 to 3975
Data columns (total 3 columns):
content    3976 non-null object
title      3976 non-null object
url        3976 non-null object
dtypes: object(3)
memory usage: 93.3+ KB


In [576]:
import pickle
# with open('powder_room_list.txt', 'wb') as f:
#     pickle.dump(df_doc_list, f)
with open('lemon_terace_list.txt', 'wb') as f:
    pickle.dump(df_doc_list, f)
# with open('moms_holic_list.txt', 'wb') as f:
#     pickle.dump(df_doc_list, f)    
    
# with open('powder_room_list.txt', 'rb') as f:
#     df_doc_list = pickle.load(f)

## 수집 대상 게시글 선정

In [577]:
# URL 정규화
df_doc_list.url.replace(to_replace='&query\S+', value='', regex=True, inplace=True)

# 중복 제거 
df_doc_list.drop_duplicates(subset=['url'], inplace=True)

# 스팸 제거
spam_keywords = '''중고차|자전거|스튜디오|기아체험|※레몬테라스 카페 체험단 / 이벤트 ♥|쏘울튜버|삼천리 쏘울|쏘울친구|쏘울티비|쏘울그라프|쏘울 충만|쏘울하우스|렌트|루이비통 티볼리|똔따렐리|한화티볼리|해운대티볼리|한화|홈쇼핑|장농|한샘|모노그램|루비똥|스피디|루이비|루이비통|루이비똥|티볼리 농장|한샘|면세점|뤼비통|티볼리gm|한X|침실|미니바이크|유모차미니버기|미니버기|무료체험|소울메이트|신청하셨어요|특약|쏘울 코트|쏘울넘치게|쏘울메이크업|핫쏘울|맵나요?|직거래|쏘울맘|출장메이크업|프리쏘울|메이크업|아기침대|쏘울맘|코나 목|코나 눈|코나 심하면|코나 귀에|코나귀로|코나 잎이|커피|코나 턱|코나 가래|마른코나|따라코나|코나 기침|코나입|코나 입|코나야|코나고 해서|코나와|코나 피|소화기|병원|기침|감기|가래|하와이|조리원|My Soul Modern Talkin|your soul|체험단|태명|음악적인 쏘울|핸드폰|쏘울양|쏘울스페셜|쏘울메이트|음악적 쏘울|아기 이름|소울에쉬|소울 1대|쏘울 가디건|쏘울 메이트|쏘울 멜란지 코트|so wool|자동차 보험|보험|쏘울 패딩|쏘울테라피|쏘울어택|쏘울푸드|쏘울여자|핫쏘울|브라|알파카|가디건|코트|쏘울블라썸|아임쏘울|아이뽀의 쏘울|다이어트|공동구매|더샘|랑콤|바이오코나|무료벼룩|에스쁘아|코나드|벼룩시장|배송비|택배|안전거래'''

df_doc_list = df_doc_list[~df_doc_list.title.str.contains(spam_keywords)]
df_doc_list = df_doc_list[~df_doc_list.content.str.contains(spam_keywords)]

# 인덱스 재설정
df_doc_list.reset_index(drop=True, inplace=True)
df_doc_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 3 columns):
content    2016 non-null object
title      2016 non-null object
url        2016 non-null object
dtypes: object(3)
memory usage: 47.3+ KB


### 게시글 및 댓글 수집

In [578]:
from pymongo import MongoClient
from datetime import datetime

# MongoDB 클라이언트 생성
client = MongoClient('mongodb://10.8.25.161:27017/')
db = client['social_data']

def parsing_target(url):
    title = driver.find_element_by_css_selector('h2.tit').text
    content = driver.find_element_by_css_selector('div#postContent').text
    author = driver.find_element_by_css_selector('span.end_user_nick').text
    post_dt = driver.find_element_by_css_selector('span[class="date font_l"]').text
    view_cnt = driver.find_element_by_css_selector('span[class="no font_l"]').text[3:]
    
    # 댓글 전체보기 클릭
    try:
        driver.find_element_by_css_selector('div.section_comment > div.top_area > a').click()
        time.sleep(5)
    except NoSuchElementException as e:
        pass

    reply_list = []    
    # 댓글 파싱 (2가지 버전이 있어 둘다 처리, 삭제된 댓글이 있어 에러처리함)
    if(driver.find_elements_by_css_selector('div.section_comment > ul > li')):
        for reply in driver.find_elements_by_css_selector('div.section_comment > ul > li'):
            try:
                reply_list.append({'reply':reply.find_element_by_css_selector('p.txt').text, 
                'reply_author':reply.find_element_by_css_selector('span[class="name ellip"]').text,
                'reply_dt': reply.find_element_by_css_selector('span.date').text})
            except NoSuchElementException:
                pass

    else:
        for reply in driver.find_elements_by_css_selector('ul.u_cbox_list > li'):
            try:
                reply_list.append({'reply':reply.find_element_by_css_selector('div.u_cbox_text_wrap').text, 
                'reply_author':reply.find_element_by_css_selector('a.u_cbox_name').text,
                'reply_dt': reply.find_element_by_css_selector('span.u_cbox_date').text})
            except NoSuchElementException:
                pass        

    result_dict = {'url': url, 'title': title, 'view_cnt': view_cnt, 'author': author, 
                   'post_dt': post_dt, 'content': content, 'replies':reply_list}
    print('%s [%s] >> %s replies crawled' % (driver.current_url, title[:20], len(reply_list)))
    return result_dict

# Mongodb 저장
def inserting_db(result_dict):
    if not(tbl.find_one({'url':result_dict['url']})):
        tbl.insert_one(result_dict)
        print('%s [%s] inserted' % (result_dict['url'], result_dict['title'][:20]))
    else:
        print('%s [%s] already in DB' % (result_dict['url'], result_dict['title'][:20]))

In [580]:
starter(cafe_ids['레몬테라스'])

print('crawling started at %s' % str(datetime.now())[:-7])

# 적재할 테이블 설정
tbl = db['lemon_terace']

error_urls = []
for index, row in df_doc_list.iterrows():    
#     if index < 258 : continue
    print('[%s/%s]' % (index+1, df_doc_list.index.max()))
    
    try:
        # 기존 수집 여부 확인
        if(tbl.find_one({'url':row['url']})):
            print('%s already in DB' % row['url'])
            continue
            
        # 수집대상 페이지로 이동
        driver.get(row['url'])
        print('connecting to %s' % row['url'])
        time.sleep(5)

        # 댓글 파싱
        result_dict = parsing_target(row['url'])
        # DB 적재
        inserting_db(result_dict)
    except NoSuchElementException as e:
        print(e)
        error_urls.append(driver.current_url)
print('crawling completed at %s' % str(datetime.now())[:-7])
driver.close()        

crawling started at 2018-01-09 13:38:40
[1/2015]
connecting to https://m.cafe.naver.com/ArticleRead.nhn?clubid=10298136&menuid=23&articleid=21801429
https://m.cafe.naver.com/CommentView.nhn?search.clubid=10298136&search.articleid=21801429&page=&sc=&search.menuid=23 [기아 소울 디젤 사고싶어용!] >> 1 replies crawled
https://m.cafe.naver.com/ArticleRead.nhn?clubid=10298136&menuid=23&articleid=21801429 [기아 소울 디젤 사고싶어용!] inserted
[2/2015]
connecting to https://m.cafe.naver.com/ArticleRead.nhn?clubid=10298136&menuid=23&articleid=20247887
https://m.cafe.naver.com/CommentView.nhn?search.clubid=10298136&search.articleid=20247887&page=&sc=&search.menuid=23 [기아 소울 자동차 어떤가요!?] >> 2 replies crawled
https://m.cafe.naver.com/ArticleRead.nhn?clubid=10298136&menuid=23&articleid=20247887 [기아 소울 자동차 어떤가요!?] inserted
[3/2015]
connecting to https://m.cafe.naver.com/ArticleRead.nhn?clubid=10298136&menuid=152&articleid=19188558
Message: Unable to locate element: h2.tit

[4/2015]
connecting to https://m.cafe.naver.com/A

In [558]:
df_doc_list.loc[258,'url']

'https://m.cafe.naver.com/ArticleRead.nhn?clubid=10094499&menuid=9&articleid=13164218'

In [2]:
from pymongo import MongoClient
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException

# MongoDB 클라이언트 생성
client = MongoClient('mongodb://10.8.25.161:27017/')
db = client['social_data']
tbl = db['youtube']

In [6]:
kk = []
for doc in db['powder'].find():
    kk.append({'title':doc['title'], 'content':doc['content'], 'post_dt':doc['post_dt']})

In [7]:
import pandas as pd
df=pd.DataFrame(kk)

In [8]:
df.head()

,content,post_dt,title
0,\n예전에 한~~~~~~~~~참 셔플 유행할때\n미국에서 이 광고로 빵 떠가지고 ...,2014.08.27. 20:01,기아 소울 햄스터 셔플 기억나세요?ㅋㅋㅋㅋㅋㅋㅋ
1,기아 야심찬 기대작이라고 하는데\n이차 정말 너무 고급스러우면서도 이쁘네요!\...,2009.06.21. 21:16,이 차 보셨나요? 와~~
2,전 갈팡질팡해서....\n룸메님들의 안목이 필요해요\n쏘울 전기차 당첨되서 사는데\...,2017.10.17. 15:31,차사는데요 색상 좀골라주세요!!!
3,이번에 중고로 쏘울 구매하려고하는데\n혹시 자동차세 얼마나 나오는줄 알수있을까요?ㅜ...,2017.01.06. 10:04,혹시 쏘울 타시는 분들 계신가요?!!
4,내년3월에 애기 출산할 예정인데\n제가타던차가 쏘울인데 오래됐고 승차감이 쎄서\n막...,2016.11.14. 18:39,여성운전자 차량 추천 부탁드려요


In [9]:
df['post_dt'] = pd.to_datetime(df['post_dt'])
df.dtypes

content            object
post_dt    datetime64[ns]
title              object
dtype: object

In [11]:
df['post_dt2'] = df['post_dt'].dt.strftime('%Y-%m-%d')

In [12]:
df.head()

,content,post_dt,title,post_dt2
0,\n예전에 한~~~~~~~~~참 셔플 유행할때\n미국에서 이 광고로 빵 떠가지고 ...,2014-08-27 20:01:00,기아 소울 햄스터 셔플 기억나세요?ㅋㅋㅋㅋㅋㅋㅋ,2014-08-27
1,기아 야심찬 기대작이라고 하는데\n이차 정말 너무 고급스러우면서도 이쁘네요!\...,2009-06-21 21:16:00,이 차 보셨나요? 와~~,2009-06-21
2,전 갈팡질팡해서....\n룸메님들의 안목이 필요해요\n쏘울 전기차 당첨되서 사는데\...,2017-10-17 15:31:00,차사는데요 색상 좀골라주세요!!!,2017-10-17
3,이번에 중고로 쏘울 구매하려고하는데\n혹시 자동차세 얼마나 나오는줄 알수있을까요?ㅜ...,2017-01-06 10:04:00,혹시 쏘울 타시는 분들 계신가요?!!,2017-01-06
4,내년3월에 애기 출산할 예정인데\n제가타던차가 쏘울인데 오래됐고 승차감이 쎄서\n막...,2016-11-14 18:39:00,여성운전자 차량 추천 부탁드려요,2016-11-14
